In [15]:
import os
import numpy as np
import pandas as pd
import argparse
from tqdm import tqdm
from random import shuffle
from music21 import *
from glob import glob
from keras.models import Sequential, Model, load_model
from keras.layers import Input, Dense, LSTM, RepeatVector, Dropout
from keras.utils import np_utils

%run lstm_model.ipynb
%run data_preprocess.ipynb
%run generate_music.ipynb

# configure.run()
# environment.set('musescoreDirectPNGPath', 'C:\\Program Files (x86)\\MuseScore 2\\bin\\MuseScore.exe')

np.set_printoptions(threshold=1000000)
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)
bach_chorale_path = 'datasets/bach2'
parts_index = [0,1,2,3]
part_num = 4 
#parts_index = [0]
note_type = set()
"""
note_type:
0-127 : note pitch
128: slur-1st-part
129: slur-2nd-part
130: slur-3rd-part
131: slur-4th-part
132: rest
133: start symbol
134: end symbol
"""

def filter_files(files):
    file_list = []
    file_count = 0
    for i, file in enumerate(files, 1):
        s = converter.parse(file)
        if len(s.parts) == part_num: # part_num = 4
            print(i, file)
            file_count += 1
            file_list.append(file)
    return file_list, file_count

def make_note_type_list():
    note_type_list = []
    for i in note_type:
        note_type_list.append(i)
    return sorted(note_type_list)

def index_note_transform(inputs, option, part_number):
    transformed_inputs = []
    if option == "to_note": 
        note_dict = dict((index, note) for index, note in enumerate(note_type))
    elif option == "to_index":
        note_dict = dict((note, index) for index, note in enumerate(note_type))

    for p_index in range(part_number):
        transformed_part = []
        for n in range(inputs.shape[1]):
            for keys in note_dict:
                if option == "to_note":# decode
                    if inputs[p_index][n] == keys:
                        transformed_part.append(note_dict[keys])
                elif option == "to_index":# encode
                    if frozenset(inputs[p_index][n]) == keys:
                        transformed_part.append(note_dict[keys])
        transformed_inputs.append(transformed_part)
    return np.array(transformed_inputs)
def notes_to_piano_roll(dataset_notes):
    note_type.add(frozenset([133]))
    note_type.add(frozenset([134]))
    for chorale_index in range(len(dataset_notes)):
        chorale = np.array(dataset_notes[chorale_index])

        chorale = index_note_transform(chorale, "to_index", len(parts_index))
        piano_roll_notes = np.transpose(chorale)
        yield piano_roll_notes
def notes_to_onehot(notes):
    note_dict = dict((index, note) for index, note in enumerate(note_type))
    onehot = []
    for i, n in enumerate(notes):
        l = []
        for note in n:
            l.append(np.array(note == np.arange(0, len(note_dict)), dtype=np.int32))
        onehot.append(l)
    return np.array(onehot)
def part_notes_to_onehot(notes):
    note_dict = dict((index, note) for index, note in enumerate(note_type))
    onehot = []
    for i, n in enumerate(notes):
        l = np.zeros(len(note_dict,), dtype=np.int32)
        for note in n:
            l[note] = 1
        onehot.append(l)
    return np.array(onehot)

def split_data(chorale_list):
    X_train = chorale_list[0]
    X_test = chorale_list[int(len(chorale_list) * 0.8)]
    notes = chorale_list[0]
    
    for i, n in enumerate(chorale_list):
        if i != 0:
            notes = np.concatenate((notes, n))
            if i < int(len(chorale_list) * 0.8):
                X_train = np.concatenate((X_train, n))
            elif i > int(len(chorale_list) * 0.8):
                X_test = np.concatenate((X_test, n))
    print(X_train.shape, X_test.shape, notes.shape)
    
    X_train_onehot_notes = part_notes_to_onehot(X_train)
    X_test_onehot_notes = part_notes_to_onehot(X_test)
    Y_train_onehot_notes = notes_to_onehot(X_train)
    Y_test_onehot_notes = notes_to_onehot(X_test)
    onehot_notes = part_notes_to_onehot(notes)
    
    return X_train_onehot_notes, X_test_onehot_notes, Y_train_onehot_notes, Y_test_onehot_notes, onehot_notes
def load_files():
    if os.path.exists(bach_chorale_path):
        file_list, num_file = filter_files(glob(bach_chorale_path + '/*.mxl') +
                                 glob(bach_chorale_path + '/*.mid'))
        return file_list, num_file
    else:
        print("The file path is wrong!!")

def fill_list(chords):
    if len(chords) < len(parts_index):
        for i in range(len(parts_index) - len(chords)):
            chords.append(frozenset([128]))
    return chords

def analyze_harmony(chorale_list, chorale_key_list):
    chorale_list = np.array(chorale_list)
    not_note_list = [128, 129, 130, 131, 132, 133, 134]
    chord_type_list = [] # Ex: I, VI, ii
    chord_list = [] 
    #print(len(chorale_key_list),chorale_key_list)
    for i, chorale in enumerate(chorale_list):
        chorale_notes = index_note_transform(np.array(chorale), "to_note", len(chorale))
        
        chorale_notes_list = []
        chord_type = []
        for j, t_notes in enumerate(chorale_notes): # Iterate over each time unit
            note_list = []
            
            for part_notes in t_notes: # Iterate over notes in each part 
                for n in part_notes:
                    if n not in not_note_list:
                        note_list.append(n)
            if len(note_list) != 0:
                _chord = chord.Chord(note_list)
                chorale_key = chorale_key_list[i]
                rf = roman.romanNumeralFromChord(_chord, chorale_key) # [chord, key]
                print(rf.figure, chorale_key.tonicPitchNameWithCase)
                chord_type.append({chorale_key.tonicPitchNameWithCase : rf.figure})
                # tonicPitchNameWithCase: Return the pitch name as a string with the proper case (upper = major; lower = minor)
                chord_degree_and_key = roman.RomanNumeral(rf.figure, chorale_key.tonicPitchNameWithCase)
                chords = [frozenset([c.midi]) for c in chord_degree_and_key.pitches]
                chords = fill_list(chords)
                if len(chords) == len(parts_index):
                    chord_list.append(chords)
                
                #print(chords)
                #display_notes.show()
            chorale_notes_list.append(note_list)
        chord_type_list.append(chord_type)
        chorale_list[i] = np.array(chorale_notes_list)
    chord_list = np.transpose(chord_list)    
    print(chord_list.shape)
    print(chord_list)
    midi = chorale_to_midi(chorale_list)
    chord_list = index_note_transform(np.array(chord_list), "to_index", len(parts_index))
    
    output_file = 'TEST.mid'
    make_midi_file(output_file, midi)

def main():
    train = True
    file_list, num_file = load_files()
    print(num_file)
    dataset_notes, chorale_key_list = make_dataset(file_list)
    shuffle(dataset_notes)
    chorale_list = [notes for chorale_index, notes in enumerate(notes_to_piano_roll(dataset_notes))]
   
    #analyze_harmony(chorale_list, chorale_key_list)
    X_train_onehot_notes, X_test_onehot_notes, Y_train_onehot_notes, Y_test_onehot_notes, onehot_notes = split_data(chorale_list)
    epochs = 50
    if train:
        for i in range(len(parts_index)): 
            lstm_model = LSTM_Model(X=X_train_onehot_notes,Y=Y_train_onehot_notes,
                                    num_units=64, epochs=epochs, batch_size=512, name="bach"+str(i))
            gen = ((X,Y) for (X, Y) in lstm_model.generator(X_train_onehot_notes, Y_train_onehot_notes, i))
            test_gen = ((X,Y) for (X,Y) in lstm_model.generator(X_test_onehot_notes, Y_test_onehot_notes, i))
            lstm_model.create_model()
            lstm_model.train_model(gen, test_gen)
    print(onehot_notes.shape)
    score = generate_notes(X_test_onehot_notes, note_type)
                
    output_file = str(epochs) + '-epochs_' + str(num_file) + '-samples.mid'
    make_midi_file(output_file, score)
if __name__ == "__main__":
    main()

1 datasets/bach2\bach_21.mid
2 datasets/bach2\bach_22.mid
3 datasets/bach2\bach_23.mid
5 datasets/bach2\bach_25.mid
6 datasets/bach2\bach_26.mid
7 datasets/bach2\bach_27.mid
9 datasets/bach2\bach_289.mid
10 datasets/bach2\bach_29.mid
11 datasets/bach2\bach_290.mid
12 datasets/bach2\bach_291.mid
13 datasets/bach2\bach_292.mid
14 datasets/bach2\bach_293.mid
15 datasets/bach2\bach_294.mid
16 datasets/bach2\bach_295.mid
17 datasets/bach2\bach_296.mid
18 datasets/bach2\bach_297.mid


Exception ignored in: <function WeakSet.__init__.<locals>._remove at 0x000001FED7DDCD08>
Traceback (most recent call last):
  File "C:\Anaconda3\lib\_weakrefset.py", line 39, in _remove
    self = selfref()
KeyboardInterrupt


19 datasets/bach2\bach_298.mid
20 datasets/bach2\bach_299.mid
22 datasets/bach2\bach_300.mid
23 datasets/bach2\bach_301.mid
24 datasets/bach2\bach_302.mid
25 datasets/bach2\bach_303.mid
26 datasets/bach2\bach_304.mid
27 datasets/bach2\bach_305.mid
28 datasets/bach2\bach_306.mid
29 datasets/bach2\bach_307.mid
30 datasets/bach2\bach_308.mid
31 datasets/bach2\bach_309.mid
32 datasets/bach2\bach_31.mid
33 datasets/bach2\bach_310.mid
34 datasets/bach2\bach_311.mid
35 datasets/bach2\bach_312.mid
36 datasets/bach2\bach_313.mid
37 datasets/bach2\bach_315.mid
38 datasets/bach2\bach_316.mid
39 datasets/bach2\bach_317.mid
40 datasets/bach2\bach_318.mid
41 datasets/bach2\bach_319.mid
43 datasets/bach2\bach_320.mid
44 datasets/bach2\bach_321.mid
45 datasets/bach2\bach_322.mid
46 datasets/bach2\bach_323.mid
47 datasets/bach2\bach_324.mid
48 datasets/bach2\bach_325.mid
49 datasets/bach2\bach_326.mid
50 datasets/bach2\bach_327.mid
51 datasets/bach2\bach_328.mid
52 datasets/bach2\bach_329.mid
54 datase

100%|██████████████████████████████████████████████████████████████████████████████████| 74/74 [01:11<00:00,  1.61it/s]


148
(34120, 4) (8312, 4) (42432, 4)
create model...
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_33 (LSTM)               (None, 16, 64)            29184     
_________________________________________________________________
dropout_61 (Dropout)         (None, 16, 64)            0         
_________________________________________________________________
lstm_34 (LSTM)               (None, 16, 64)            33024     
_________________________________________________________________
dropout_62 (Dropout)         (None, 16, 64)            0         
_________________________________________________________________
dense_26 (Dense)             (None, 16, 49)            3185      
Total params: 65,393
Trainable params: 65,393
Non-trainable params: 0
_________________________________________________________________
train model...
Epoch 1/65
34s - loss: 1.5379 - acc: 0.6858 - val_loss: 1.0724 - val_ac

Epoch 11/65
18s - loss: 1.0001 - acc: 0.7229 - val_loss: 0.9021 - val_acc: 0.7109
Epoch 12/65
19s - loss: 1.0295 - acc: 0.7050 - val_loss: 1.0001 - val_acc: 0.7031
Epoch 13/65
18s - loss: 0.9980 - acc: 0.7152 - val_loss: 1.0225 - val_acc: 0.6797
Epoch 14/65
18s - loss: 1.0145 - acc: 0.7174 - val_loss: 0.9950 - val_acc: 0.7070
Epoch 15/65
19s - loss: 0.9771 - acc: 0.7266 - val_loss: 0.9853 - val_acc: 0.7031
Epoch 16/65
19s - loss: 1.0311 - acc: 0.7052 - val_loss: 0.6861 - val_acc: 0.8711
Epoch 17/65
18s - loss: 0.9594 - acc: 0.7211 - val_loss: 0.9510 - val_acc: 0.7266
Epoch 18/65
18s - loss: 1.0044 - acc: 0.7179 - val_loss: 0.9115 - val_acc: 0.7266
Epoch 19/65
18s - loss: 0.9309 - acc: 0.7371 - val_loss: 0.9419 - val_acc: 0.7031
Epoch 20/65
19s - loss: 1.0212 - acc: 0.7043 - val_loss: 1.1536 - val_acc: 0.6836
Epoch 21/65
17s - loss: 0.9346 - acc: 0.7273 - val_loss: 0.9731 - val_acc: 0.7031
Epoch 22/65
17s - loss: 1.0136 - acc: 0.7106 - val_loss: 1.1562 - val_acc: 0.6719
Epoch 23/65
17s 

KeyboardInterrupt: 